To je zdej ista koda kot za male grafe samo da racuna se geodesic number

In [1]:
import itertools
from sage.all import *
import pandas as pd
import os
from multiprocessing import Pool, cpu_count

# -------------------------------------------------
# CYCLOMATIC NUMBER
# -------------------------------------------------
def cyclomatic_number(G):
    return G.size() - G.order() + G.connected_components_number()

# -------------------------------------------------
# GEODESIC SUBPATH NUMBER
# -------------------------------------------------
def geodesic_subpath_number(G):
    V = list(G.vertices())
    idx = {v: i for i, v in enumerate(V)}
    n = len(V)

    dist = G.distance_all_pairs()
    counts = [[0] * n for _ in range(n)]

    for s in V:
        s_idx = idx[s]

        def dfs(u, visited, length):
            u_idx = idx[u]

            if length == dist[s][u]:
                counts[s_idx][u_idx] += 1
            else:
                return

            for nei in G.neighbors(u):
                if nei not in visited:
                    visited.add(nei)
                    dfs(nei, visited, length + 1)
                    visited.remove(nei)

        visited = {s}
        dfs(s, visited, 0)

    total = 0
    for i in range(n):
        for j in range(i, n):
            total += counts[i][j]

    return total

# -------------------------------------------------
# OBDELAVA PARA (μ, n)
# -------------------------------------------------
def procesiraj_mu_n(param):
    mu_target, n = param

    kandidati = []

    for G in graphs(n):
        if not G.is_connected():
            continue

        mu = cyclomatic_number(G)
        if mu != mu_target:
            continue

        gnum = geodesic_subpath_number(G)

        kandidati.append((gnum, G))

    if len(kandidati) == 0:
        return None

    kandidati.sort(key=lambda x: x[0])

    min_g, G_min = kandidati[0]
    max_g, G_max = kandidati[-1]

    min_slika = f"slike_geodesic_min/graf_geodesic_min_mu{mu_target}_n{n}.png"
    max_slika = f"slike_geodesic_max/graf_geodesic_max_mu{mu_target}_n{n}.png"

    G_min.plot().save(min_slika)
    G_max.plot().save(max_slika)

    return [
        n,
        mu_target,
        min_g,
        max_g,
        min_slika,
        max_slika,
        G_min.graph6_string(),
        G_max.graph6_string()
    ]

# -------------------------------------------------
# PARALELNE NALOGE
# -------------------------------------------------

MUJEV = range(1, 9)
N_RANGE = range(3, 10)

tasks = [(mu, n) for mu in MUJEV for n in N_RANGE]

os.makedirs("slike_geodesic_min", exist_ok=True)
os.makedirs("slike_geodesic_max", exist_ok=True)

print(f"Uporabljam {cpu_count()} CPU jeder ...")

with Pool(cpu_count()) as p:
    results = p.map(procesiraj_mu_n, tasks)

results = [r for r in results if r is not None]

# -------------------------------------------------
# CSV
# -------------------------------------------------

df = pd.DataFrame(
    results,
    columns=[
        "n",
        "µ(G)",
        "min geodesic",
        "max geodesic",
        "slika_min",
        "slika_max",
        "graph6_min",
        "graph6_max"
    ]
)

df.to_csv("geodesic_mu_mali_grafi.csv", index=False)

print("\n--- IZRAČUN KONČAN ---")
print("Vsi CPU so bili uporabljeni.")


Uporabljam 8 CPU jeder ...

--- IZRAČUN KONČAN ---
Vsi CPU so bili uporabljeni.
